##        **Práctica** **2** (preguntas 1 y 2):
El siguiente programa pretende simular un sistema, para obtener estadísticas de una clínica del IMSS. Evaluaremos mediante dos políticas de atención a los usuarios. Se tomó la decisión de realizar el proceso de espera por medio de una simulación de colas. Tomamos en cuenta ciertos parámetros para la elaboración de la simulación estadística.
Las dos potíticas de atención que realizamos fueron: (1) "Primero en llegar primero en salir" y (2) "Prioridad a pacientes de urgencia sobre pacientes rutinarios".

In [ ]:
#CÓDIGO PARA PREGUNTA 1 Y 2
import numpy as np


class Personas():
    """La clase personas se encarga de almacenar los metodos de todas las
    personas definiendolas como "objetos", es decir, no dira el tipo de paciente que va a ser
    la persona, su probabilidad de uniforme del tiempo interarribo, y su tiempo
    de espera """
    def __init__(self, t_llegada, t_salida):
        """En este metodo se creara a la personas o usuarios como "objetos", los
        cuales tendran como atributo un tiempo de llegada y uno de salida """
        self.t_llegada = t_llegada
        self.t_salida = t_salida
        self.t_paciente = 0 if np.random.random() < 0.2 else 1
        #Se genera al tipo de paciente de acuerdo a una probabilidad de llegada establecida del 20%

    def uniforme(self, a, b):
        """En este metodo se define la uniformidad en el intervalo [0.5, 50] por
        medio de una función que mandara al intervalo [0,1] al intervalo [0.5, 50]
        es decir crea o generas un número aleatorio dentro del intervalo de llegada"""
        u = (b-a)*np.random.random() + a
        return u

    def interarribo(self):
        """Regresa una probabilidad uniforme en todo el intervalo [0.5, 50]
        para gernerar los tiempos interarribo entre personas"""
        return self.uniforme(0.5, 50)

    def tiempo_espera(self):
        """Se define y calcula al tiempo de espera del "objeto" persona por
        medio de sus atributos"""
        return self.t_salida - self.t_llegada

class Cola():
    """"La clase cola se encarga de hacer la simulacion de las colas o filas de
    los pacientes o usuarios"""
    def __init__(self):
        """Se construye la fila que va acumular a los usuarios o personas"""
        self.lista = []

    def enqueue_persona(self, persona):
        """Se encarga de agregar a los pacientes que vayan llegando al sistema"""
        self.lista.append(persona)

    def desenqueue_persona(self):
        """Se encarga de quitar siempre a la primer persona de la fila, ya que
        vaya a entrar al consultorio"""
        return self.lista.pop(0)

    def is_empty(self):
        """Se encarga de revisar si la fila es vacia o no por medio de un bool """
        return True if len(self.lista) == 0 else False

    def top(self):
        """Siempre va revisar al primer usuario de la fila"""
        return self.lista[0]

class Simulacion():
    """Clase simulacion se encarga de simular la llegada, salida del consultorio
    y de la fila de 50000 usuarios, los cuales utilizan distintos metodos, para
    generar la estadistica requerida"""
    def __init__(self, n_personas):
        """Va construir la simulacion con el numero requerido de personas"""
        self.n_personas = n_personas
        self.lista = Cola()
        self.reloj = 0
        self.t_consulta = 20
        self.tiempo_espera = []

    def evento_llegada(self):
        """Simula la llegada de las peronas al consultorio"""
        t_interarribo = self.lista.top().interarribo() if not self.lista.is_empty() else Personas(self.reloj, None).interarribo()
        self.reloj += t_interarribo
        #Actualizara la hora de llegada de la personan respecto a la hora de llegada de la persona n-1 + es tiempo interarribo
        personita = Personas(self.reloj, None)
        self.lista.enqueue_persona(personita)

    def evento_salida_cola(self):
        """Simula la salida de un usuario o personas de la cola o fila"""
        persona = self.lista.top()
        if persona:
            persona = self.lista.desenqueue_persona()
            persona.t_salida = self.reloj
             #Actualizara la hora de salida de la persona de la fila o cola
            self.tiempo_espera.append(persona.tiempo_espera())
        return persona

    def evento_salida_consultorio(self, persona):
        """Simula la salida de un usuario o personas pero ahora del sistema"""
        persona.t_salida += self.t_consulta
        return persona

    def aux_reiniciar_simulacion(self):
        """Metodo auxiliar para reiniciar la simulacion para cada iteracion del
        simulador FIFO"""
        self.reloj = 0
        self.lista = Cola()
        self.tiempo_espera = []

    def simulador_FIFO(self, n_simulaciones):
        """Se encarga de realizar la simulacion para los tiempos de espera promedio
        y los tiempos prodio del sistema"""
        tiempo_promedio_espera_t = 0
        tiempo_promedio_sistema_t = 0
        #Se crean 2 nuevos contadores para acumular los tiempos promedio de espera y de sistema.

        for _ in range(n_simulaciones):
          #realiza el número de simulaciones indicadas
            for _ in range(self.n_personas):
              #empieza la llegada de las personas requeridas para la estadistica
                self.evento_llegada()

            while not self.lista.is_empty():
              # Mientras la fila o cola no este vacía, va a procesar a las personas en la cola hasta que esté vacía.
                persona = self.evento_salida_cola()
                persona = self.evento_salida_consultorio(persona)

            tiempo_promedio_espera_t += np.mean(self.tiempo_espera)
            tiempo_promedio_sistema_t += np.mean(self.tiempo_espera) + self.t_consulta

            self.aux_reiniciar_simulacion()

        tiempo_promedio_espera = tiempo_promedio_espera_t / n_simulaciones
        tiempo_promedio_sistema = tiempo_promedio_sistema_t / n_simulaciones

        return tiempo_promedio_espera, tiempo_promedio_sistema

# Realiza las  20 simulaciones
simulacion = Simulacion(n_personas=50000)
tiempo_promedio_espera, tiempo_promedio_sistema = simulacion.simulador_FIFO(n_simulaciones=20)

print("Tiempo promedio de espera de los pacientes:", tiempo_promedio_espera)
print("Tiempo promedio en el sistema:", tiempo_promedio_sistema)

Tiempo promedio de espera de los pacientes: 631499.8611141796
Tiempo promedio en el sistema: 631519.8611141796


            
##        **Práctica** **2** (pregunta 3):


El siguiente programa es el que se usó anteriormente, sin embargo, modificamos el código ligeramente para darle respuesta a la pregunta número 3 de la Práctica 2.

In [ ]:
#CÓDIGO PARA PREGUNTA 3
import numpy as np

class Personas():
    """La clase personas se encarga de almacenar los metodos de todas las
    personas definidas como "objetos", es decir, no dira el tipo de paciente que va a ser
    la persona, su probabilidad de uniforme del tiempo interarribo, y su tiempo
    de espera """
    def __init__(self, t_llegada, t_salida):
        """En este metodo se creara a la personas o usuarios como "objetos", los
        cuales tendran como atributo un tiempo de llegada y uno de salida """
        self.t_llegada = t_llegada
        self.t_salida = t_salida
        self.t_paciente = 0 if np.random.random() < 0.2 else 1

    def uniforme(self, a, b):
        """En este metodo se define la uniformidad en el intervalo [0.5, 50] por
        medio de una funcion que mandara al intervalo [0,1] al intervalo [0.5, 50]
        es decir crea o generas un número aleatorio dentro del intervalo de llegada"""
        u = (b - a) * np.random.random() + a
        return u

    def interarribo(self):
        """Regresa una probabilidad uniforme en todo el intervalo [0.5, 50]
        para generar los tiempos interarribo entre personas"""
        return self.uniforme(0.5, 50)

    def tiempo_espera(self):
        """Se define y calcula al tiempo de espera del "objeto" persona por
        medio de sus atributos"""
        return self.t_salida - self.t_llegada

class Cola():
    """"La clase cola se encarga de hacer la simulacion de las colas o filas de
    los pacientes o usuarios"""
    def __init__(self):
        """Se construye la fila que va acumular a los usuarios o personas"""
        self.lista = []

    def enqueue_persona(self, persona):
        """Se encarga de agregar a los pacientes que vayan llegando al sistema"""
        self.lista.append(persona)

    def desenqueue_persona(self):
        """Se encarga de quitar siempre a la primer persona de la fila, ya que
        vaya  a entrar al consultorio"""
        return self.lista.pop(0)

    def is_empty(self):
        """Se encarga de revisar si la fila es vacia o no por medio de un bool """
        return len(self.lista) == 0

    def top(self):
        """Siempre va revisar al primer usuario de la fila"""
        return self.lista[0]

class Simulacion():
    """Clase simulacion se encarga de simular la llegada, salida del consultorio
    y de la fila de 50000 usuarios, los cuales utilizan distintos metodos, para
    generar la estadistica requerida"""
    def __init__(self, n_personas):
        """Va construir la simulacion con el numero requerido de personas"""
        self.n_personas = n_personas
        self.lista = Cola()
        self.reloj = 0
        self.t_consulta = 20
        self.tiempo_espera = []
        self.tiempo_espera_u = []
        self.tiempo_espera_r = []
        self.pacientes_r = Cola()
        self.pacientes_u = Cola()

    def evento_llegada(self):
        """Simula la llegada de las peronas al consultorio, pero, ahora se
        agregan por tipo de paciente, actualizando la hora de llegada del paciente
        de la cola"""
        t_interarribo = self.lista.top().interarribo() if not self.pacientes_u.is_empty() else Personas(self.reloj, None).interarribo()
        self.reloj += t_interarribo
        personita = Personas(self.reloj, None)
        self.lista.enqueue_persona(personita)

        if personita.t_paciente == 0:
            self.pacientes_u.enqueue_persona(personita)
        else:
            self.pacientes_r.enqueue_persona(personita)

    def evento_salida_cola(self):
        """Simula la salida de un usuario o personas de la cola o fila, al igual
        que el metodo anterior, simula la salida de la cola por tipo de paciente
        actualizando ahora la hora de salida del paciente de la cola"""
        personita = None          #para manejar los casos donde no hay personitas en la cola o fila para elaborar el proceso.
        if not self.pacientes_u.is_empty():
            personita = self.pacientes_u.desenqueue_persona()
            personita.t_salida = self.reloj
            self.tiempo_espera_u.append(personita.tiempo_espera())
        elif not self.pacientes_r.is_empty():
            personita = self.pacientes_r.desenqueue_persona()
            personita.t_salida = self.reloj
            self.tiempo_espera_r.append(personita.tiempo_espera())
        return personita

    def evento_salida_consultorio(self, personita):
        """Simula la salida de un usuario o personas, pero, ahora del sistema"""
        if personita is not None:
         #En caso de que la cola o fila este vacia y personita sea None, el metodo devolvera None sin hacer nada, evitando error de tipo "NoneType"
            personita.t_salida += self.t_consulta
        return personita

    def aux_reiniciar_simulacion(self):
        """Metodo auxiliar para reiniciar la simulacion para cada iteracion del
        simulador prioridad"""
        self.reloj = 0
        self.lista = Cola()
        self.tiempo_espera_u = []
        self.tiempo_espera_r = []

    def simulador_prioridad(self, n_simulaciones):
        """Se encarga de realizar la simulacion para los tiempos de espera promedio
        y los tiempos promedio del sistema, pero ahora por separado es decir, por
        tipo de paciente"""
        tiempo_promedio_espera_total_u = 0
        tiempo_promedio_espera_total_r = 0

        for i in range(n_simulaciones):
            for i in range(self.n_personas):
                self.evento_llegada()

            while not self.pacientes_u.is_empty() or not self.pacientes_r.is_empty():
                if not self.pacientes_u.is_empty():
                    personita = self.evento_salida_cola()
                    personita = self.evento_salida_consultorio(personita)
                else:
                    personita = self.evento_salida_cola()
                    personita = self.evento_salida_consultorio(personita)

            tiempo_promedio_espera_total_u += np.mean(self.tiempo_espera_u)
            tiempo_promedio_sistema_total_u = np.mean(self.tiempo_espera_u) + self.t_consulta
            tiempo_promedio_espera_total_r += np.mean(self.tiempo_espera_r)
            tiempo_promedio_sistema_total_r = np.mean(self.tiempo_espera_r) + self.t_consulta

            self.aux_reiniciar_simulacion()

        tiempo_promedio_espera_u = tiempo_promedio_espera_total_u / n_simulaciones
        tiempo_promedio_sistema_u = tiempo_promedio_sistema_total_u / n_simulaciones
        tiempo_promedio_espera_r = tiempo_promedio_espera_total_r / n_simulaciones
        tiempo_promedio_sistema_r = tiempo_promedio_sistema_total_r / n_simulaciones

        return tiempo_promedio_espera_u, tiempo_promedio_sistema_u, tiempo_promedio_espera_r, tiempo_promedio_sistema_r


# Realiza las 20 simulaciones
simulacion_prioridad = Simulacion(n_personas=50000)
tiempo_promedio_espera_u, tiempo_promedio_sistema_u, tiempo_promedio_espera_r, tiempo_promedio_sistema_r = simulacion_prioridad.simulador_prioridad(n_simulaciones=20)
print("Tiempo promedio de espera con prioridad_u:", tiempo_promedio_espera_u)
print("Tiempo promedio en el sistema con prioridad_u:", tiempo_promedio_sistema_u)
print("Tiempo promedio de espera con prioridad_r:", tiempo_promedio_espera_r)
print("Tiempo promedio en el sistema con prioridad_r:", tiempo_promedio_sistema_r)

Tiempo promedio de espera con prioridad_u: 631005.5622646635
Tiempo promedio en el sistema con prioridad_u: 31310.274389107006
Tiempo promedio de espera con prioridad_r: 630757.3490218405
Tiempo promedio en el sistema con prioridad_r: 31706.738402686344
